In [1]:
import pandas as pd
from konlpy.tag import Mecab
from functools import *
from collections import Counter
import numpy as np
from tqdm.auto import tqdm

In [2]:
def get_word(res):
    res_w = []
    for word in res:
        if len(word) < 2:
            continue
        else:
            res_w.append(word)
    return res_w

tokenizer = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic")

def split_word_df(df_trg):
    res_list = []
    print(f'total : {len(df_trg)}')
    for content in tqdm(df_trg.contents_clean):
        res = tokenizer.nouns(content)
        res = get_word(res)
        res_list.append(res)
    df_trg['words'] = res_list
    return df_trg

def get_word_pct_dict(lst_word , df_src , trg_col):
    word_pct_dict = {}
    for word in tqdm(lst_word):
        word_pct_dict.update(  {word : len(df_src[df_src[trg_col].apply(lambda x: word in x)])/len(df_src)} )
        sorted_word_pct_dict = sorted(word_pct_dict.items(), key=lambda x:x[1], reverse=True)
    return sorted_word_pct_dict

def flattened_list(lst):
    flattened_lst = [item for sublist in lst for item in sublist]
    return flattened_lst

In [3]:
df = pd.read_parquet('df_qesg28_final_time.pq')

# 전체 기준으로 단어 빈도 

In [4]:
df_temp = split_word_df(df)

total : 106164


  0%|          | 0/106164 [00:00<?, ?it/s]

In [5]:
# 저장
df_temp= df_temp.reset_index(drop= False)
df_temp.to_parquet('df_temp.pq')

# Load Data

In [6]:
df_temp = pd.read_parquet('df_temp.pq')

## 전체 코퍼스 만들기

In [7]:
# loop 로 전체 코퍼스 만들기
corpus = []
for val in tqdm(df_temp.words):
    corpus += list(val)

corpus= list(set(corpus))
print(len(corpus))

  0%|          | 0/106164 [00:00<?, ?it/s]

99688


In [8]:
## 오래 걸려서 안씀
'''
# 전체에서 90% 이상 나오는 단어는 제거 
df_trg = df_temp
# 단어 리스트 
word_set = []
for i , row in tqdm(df_trg.iterrows()):
    word_set += list(set(row.words))
word_set = list(set(word_set))
    

print(len(word_set))
count_mat = np.zeros_like(word_set , dtype = 'int32')
for i , word in tqdm(enumerate(word_set)):
    for k , row in df_trg.iterrows():
        if word in row.words:
            count_mat[i] += 1 
'''

"\n# 전체에서 90% 이상 나오는 단어는 제거 \ndf_trg = df_temp\n# 단어 리스트 \nword_set = []\nfor i , row in tqdm(df_trg.iterrows()):\n    word_set += list(set(row.words))\nword_set = list(set(word_set))\n    \n\nprint(len(word_set))\ncount_mat = np.zeros_like(word_set , dtype = 'int32')\nfor i , word in tqdm(enumerate(word_set)):\n    for k , row in df_trg.iterrows():\n        if word in row.words:\n            count_mat[i] += 1 \n"

# 특정 카테고리 기준으로 단어 빈도 

In [9]:
df

,id,keyword,time,title,link,contents,y,contents_clean,words
0,5,삼성전자,2020-05-11 20:50:52,"[OA칼럼] 삼성전자, D램 EUV 공정 적용 의미와 전망은",http://www.thelec.kr/news/articleView.html?idx...,파운드리 사업과 장비 사용 공유\n\n편집자 주 : 전자부품 전문미디어 디일렉은 미...,제외,파운드리 사업과 장비 사용 공유편집자 주 전자부품 전문미디어 디일렉은 미국 메모리 ...,"[파운드리, 사업, 장비, 사용, 공유, 편집자, 전자, 부품, 전문, 미디어, 미..."
1,13,삼성전자,2020-05-11 03:51:02,"애플은 더 싸게 내놓고, 교체수요는 뚝… 위기의 삼성폰",https://news.chosun.com/site/data/html_dir/202...,삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹(대유행)에 직격탄을...,제외,삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹에 직격탄을 맞아 판...,"[삼성전자, 스마트폰, 사업, 위기감, 코로나, 팬데, 직격탄, 판매량, 급감, 최..."
2,18,삼성전자,2020-05-11 13:51:02,5주년 맞은 삼성페이의 진화... 비대면 결제시장서 '기회',http://biz.newdaily.co.kr/site/data/html/2020/...,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시S6 출시를 한달 여 앞둔 상황에...,제외,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시s6 출시를 한달 여 앞둔 상황에...,"[삼성전자, 상반기, 플래그, 갤럭시, 출시, 상황, 미국, 모바일, 결제, 솔루션..."
3,19,삼성전자,2020-05-11 08:51:02,"`경영권 불승계`…""삼성물산·전자, 기업가치 재평가 기대""",https://www.edaily.co.kr/news/read?newsId=0144...,[이데일리 김재은 기자] 미래에셋대우는 11일 삼성물산(028260) 삼성전자(00...,주주가치보호,미래에셋대우는 11일 삼성물산 삼성전자 등 삼성그룹주의 기업가치 재평가가 기대된다고...,"[미래, 대우, 삼성물산, 삼성전자, 삼성그룹, 기업, 가치, 재평가, 기대, 이재..."
4,30,삼성전자,2020-05-10 00:00:00,이건희 회장 병상 6년… 이재용표 '뉴 삼성' 도약할까,https://www.seoul.co.kr/news/newsView.php?id=2...,李 회장 여전히 의식 없지만 자가 호흡\n\n▲ 이재용 삼성전자 부회장.\n\n연합...,소유구조,회장 여전히 의식 없지만 자가 호흡 이재용 삼성전자 부회장.이건희 삼성전자 회장이 ...,"[회장, 의식, 호흡, 이재용, 삼성전자, 부회장, 이건희, 삼성전자, 회장, 병상..."
...,...,...,...,...,...,...,...,...,...
106159,350816,한국동서발전,2021-04-28 16:23:37,"한국동서발전, 일회용품 안쓰기 캠페인 실시…""환경 지키기 앞장""",https://www.mk.co.kr/news/business/view/2021/0...,\n\n임직원들이 `일회용품 안쓰기로 지구살리기` 캠페인을 시행하고 있다. [사진 ...,자원순환,임직원들이 일회용품 안쓰기로 지구살리기 캠페인을 시행하고 있다.,"[임직원, 일회용품, 지구, 캠페인, 시행]"
106160,350817,한국동서발전,2021-04-28 16:17:38,"[탄소중립 결의] ""경제성장·기후환경 두 마리 토끼 잡는다 ...",https://www.ekn.kr/web/view.php?key=2021042701...,▲이학영 국회 산업통상자원중소벤처기업위원회 위원장이 28일 서울 여의도 중소기업중앙...,에너지및온실가스,이학영 국회 산업통상자원중소벤처기업위원회 위원장이 28일 서울 여의도 중소기업중앙회...,"[이학영, 국회, 산업, 통상, 자원, 중소, 벤처, 기업, 위원회, 위원장, 서울..."
106161,350820,한국동서발전,2021-04-28 10:35:40,[시사일보] 수원시 영통구 망포2동 '미래세대를 위한 작은 실천 ...,http://www.koreasisailbo.com/274728,망포2동‘미래세대를 위한 작은 실천’캠페인 [시사일보=최승곤 기자] 수원시 영통...,제외,망포2동미래세대를 위한 작은 실천캠페인 수원시 영통구 망포2동 주민자치위원회가 지속...,"[망포, 미래, 세대, 실천, 캠페인, 수원시, 영통구, 망포, 주민, 자치, 위원..."
106162,350827,기업은행,2021-04-28 10:38:15,"IB캐피탈, 한부모 거주시설에 차량 2대 기증...소외계층 돕는다",http://www.babytimes.co.kr/news/articleView.ht...,IBK캐피탈 최현숙(왼쪽) 대표와 대한사회복지회 김석현(오른쪽) 회장(사진=IBK기...,사회공헌,ibk캐피탈 최현숙 대표와 대한사회복지회 김석현 회장 ibk기업은행의 자회사 ibk...,"[캐피탈, 최현숙, 대표, 대한, 사회, 복지, 김석현, 회장, 기업, 은행, 자회..."


In [10]:
# 기간 지정 및 제외 기타 제거 
df = df_temp
df_sm = df.loc[df.contents_clean != '']\
    .loc[df.y != '제외']\
    .loc[df.y != '기타']\
#    .loc[df.time < '2021-04-28']\
#    .loc[df.time > '2020-01-01']

In [11]:
df_sm.y.value_counts()

에너지및온실가스       8005
인적자원관리         7116
사회공헌           6735
소유구조           5893
제품/서비스         3598
ESG거버넌스        2939
공급망관리          2644
사업장안전및보건       2245
내부통제/투명성       1981
소비자보호          1929
이사회구성과운영       1852
지역사회영향         1754
공정거래           1659
자원순환           1280
계열회사와의거래        863
감사기구            637
주주가치보호          609
이익배분            484
인권              431
대기오염            251
환경관리체계          208
공시              203
수질오염            104
화학물질배출           83
친환경제품/서비스개발      79
생물다양성            54
기타오염             33
토양오염             15
Name: y, dtype: int64

## 원하는 카테고리 저장

In [12]:
df_trg = df.loc[df.y == '환경관리체계']
df_trg.to_excel('환경관리체계.xlsx') 
#df_trg = pd.read_excel('환경관리체계.xlsx', index_col= 0)

In [13]:
df_trg = df.loc[ df.y == '환경관리체계']


# 토크나이저 
- 단어별 등장 빈도로 필터링 하려 했으나, 사용하지 않음. 

In [14]:
# 전체에서 90% 이상 나오는 단어는 제거 

# 단어 리스트 
word_set = []
for i , row in df_trg.iterrows():
    word_set += list(set(row.words))

word_set = list(set(word_set))
    
print(len(word_set))
count_mat = np.zeros_like(word_set , dtype = 'int32')
for i , word in tqdm(enumerate(word_set)):
    for k , row in df_trg.iterrows():
        if word in row.words:
            count_mat[i] += 1 
        

    

4550


0it [00:00, ?it/s]

In [15]:
doc_count = len(df_trg) # total doc count 
ratio_mat = count_mat / doc_count # 뉴스 1개 등장 비율/ 전체 문서 등장 비율

In [16]:
# 단어별 TF / IDF 데이터 프레임화
df_ratio = pd.DataFrame()
df_ratio['word']  = word_set
df_ratio['ratio'] = ratio_mat

In [17]:
# 단어중 최빈도 단어의 비율 * 80 의 비율값
df_ratio_top = df_ratio.loc[df_ratio.ratio >= 0.8* df_ratio.ratio.max() ]
print(0.8* df_ratio.ratio.max())

0.5320574162679427


In [18]:
# 단어중 최빈도 단어의 비율 * 80 이상의 비율을 가진 단어 제거 
df_ratio_cuttop = df_ratio.loc[df_ratio.ratio < 0.9* df_ratio.ratio.max() ]

In [19]:
df_ratio_cuttop.sort_values(by = 'ratio', ascending=False)

,word,ratio
26,기업,0.583732
3295,경영,0.516746
3440,에너지,0.488038
2440,투자,0.473684
4395,계획,0.454545
...,...,...
1985,초석,0.004785
1986,준원,0.004785
1987,주범,0.004785
1988,고부,0.004785
